In [1]:
import pandas as pd
import numpy as np
import re
import editdistance

In [2]:
col_list= [
      "PatientID",
      ".SD",
      "CStage",
      "NumbOfBx",
      "Years",
      "Difference",
      "barplot",
      "head",
      "read.table",
      "eHospitalNum",
      "pHospitalNum",
      ".",
      "EVENT",
      "MonthYear",
      "freq",
      "Endoscopist",
      "avg",
      "v",
      "destination",
      "dcast",
      "complete.cases",
      "g",
      "gvisSankey",
      "head",
      "pHospitalNum",
      "par",
      "plot",
      "r",
      "read.table",
      "region",
      "rgb",
      "setDT",
      "Myendo",
      "Mypath",
      "doc_id",
      "sentence",
      "upos",
      "xpos",
      "feats",
      "udmodel_english",
      "cbind_dependencies",
      "head_token_id",
      "deps",
      "morph_voice",
      "OriginalReport",
      "dep_rel",
      "misc",
      "has_morph",
      "morph_abbr",
      "morph_case",
      "morph_definite",
      "morph_degree",
      "morph_foreign",
      "morph_gender",
      "morph_mood",
      "morph_number",
      "morph_numtype",
      "morph_person",
      "morph_poss",
      "morph_prontype",
      "morph_reflex",
      "morph_tense",
      "morph_verbform",
      "str_subset",
      "lower",
      "upper",
      "pathSiteString",
      "outputFinal",
      "theme_foundation",
      "udmodel",
      "x3",
      "FINDINGSmyDx",
      "FindingsAfterProcessing",
      "primDxVector",
      "Temporal",
      "sentence_id",
      "img",
      "Endo_ResultEntered",
      "V1",
      "pandoc.image.return"]

In [3]:
HistolType = {
    "Resection": "Resection",
    "bx":"Biopsy"
    "biopsies":"Biopsy",
    "endoscopic mucosal resection" = "EMR"
    "endoscopic mucosectomy" = "EMR",
    "endoscopic submucosal dissection" = "ESD",
    "endoscopic submucosal resection" = "ESD",
    "nodul":"nodul",
    "polyp":"polyp",
    "emr":"EMR" #@Leo: we will lower case all the text, why need this?
}
LocationListUpper = {
    "d2" = "Duodenum"
    "duodenal" = "Duodenum",
    "gastric" = "Stomach",
    "antral" = "Antrum",
    "ogj" = "GOJ",
    "gastrooesophageal" = "GOJ",
    "fundal|fundus" = "stomach",
    "pyloric" = "Pylorus",
    "gastric cardia" = "Cardia",
    "oesophageal" = "Oesophagus"
}



#' Use list of standard locations for upper GI endoscopy
#'
#' The is a list of standard locations at endoscopy that is used in the
#' extraction of the site of biopsies/EMRs and potentially in functions looking at the site of a
#' therapeutic event. It just returns the list in the function
#'
#'
#' @keywords Location
#' @family NLP - Lexicons
#' @export

LocationListUniversal = {"Anastomosis": "Ascending"}


#' Use list of standard locations for lower GI endoscopy
#'
#' The is a list of standard locations at endoscopy that is used in the
#' extraction of the site of biopsies/EMRs and potentially in functions looking at the site of a
#' therapeutic event. It just returns the list in the function
#'
#'
#' @keywords Location
#' @family NLP - Lexicons
#' @export

LocationListLower = {
    "asce |ascending|(ascend[^a-z])|( colon r )|(r colon)|(asc )|(right colon)" = "Ascending ",
    "descending|(descen[^a-z])|(desc[^a-z])|(des[^a-z])|(colon l)|(l colon)|(left colon)" = "Descending ",
    "sigmoid|(sigm[^a-z])|sigmo " = "Sigmoid ",
    "rectal|rectum|(rectum[a-z])|rect " = "Rectum ",
    "(transv[^a-z])|tranv |trans " = "Transverse ",
    "caecal" = "Caecum ",
    #"splenic" = "Splenic ",
    "ileal" = "Ileum ", #@Leo: space?
    #"rectosigmoid" = "Rectosigmoid ",
    "(ileocaecal\\s*)|icv|(ileo-caecum)" = "Ileocaecal",
    "(hep[^a-z])|([Hh]epatic)" = "Hepatic ",
    "colonic|colon |(col[^a-z])" = "Colon ",
    #"term |terminal" = "Terminal",
    #"TI " = "Terminal", #@Leo: can't mapping coz all being lowercased
    "ti" = "Terminal"
    "caec" = "Caecum ",
    "sig " = "Sigmoid ",
    "ileo\\s*-\\s*anal|ileo\\s*anal " = "Ileoanal ",
    "(pre\\s*pouch)|(pre-[Pp]ouch)" = "PrePouch",
    #"pouch" = "Pouch",
    "term" = "Terminal"
}

SyntaxError: invalid syntax (<ipython-input-3-bf4e227658d1>, line 4)

In [ ]:
dict_test = {'[Ss]':'s'}
dict_test['[Ss]']

In [ ]:
example = "Hospital: Random NHS Foundation Trust\nHospital Number: J6044658\nPatient Name:  Jargon, Victoria\nGeneral Practitioner: Dr. Martin, Marche\nDate of procedure:  2009-11-11\nEndoscopist: Dr. Sullivan, Shelby\nSecond endoscopist: Dr. al-Basha, Mahfoodha\nMedications: Fentanyl  12.5mcg\nMidazolam  6mg\nInstrument:  FG5\nExtent of Exam:  GOJ\nIndications: Follow-up ULCER HEALING\nProcedure Performed: Gastroscopy (OGD)\nFindings:  No evidence of Barrett's oesophagus, short 2 cn hiatus hernia.,Oesophageal biopsies taken from three levels as requested.,OGD today to assess for ulceration/ongoing bleeding.,Diaphragmatic pinch:40cm .,She has a small hiatus hernia .,We will re-book for 2 weeks, rebanding.,Tiny erosions at the antrum.,Biopsies taken from top of stricture-metal marking clips in situ.,The varices flattened well with air insufflation.,He is on Barrett's Screeling List in October 2017 at St Thomas'.\nHALO 90 done with good effect\nEndoscopic Diagnosis:  Post chemo-radiotherapy stricture "

In [ ]:
def text_prep(input_text, delim):
    #1. Flatten the text..
    input_text = lower(input_text) #@Leo: Do we want to make everything lower case?
    input_text = column_cleanup(input_text)
    

In [ ]:
def col_splitter(text, delim):
    delim = re.sub(r':',r'', delim)
    delim = re.sub(r'(.*)',r'\1: ',delim)
    delim = list(delim)
    text = dict_replacement(text, delim)
    text = re.sub( r': :',r': ', text)
    text = re.sub(r':([A-Za-z0-9])',r': \1',text)
    text = re.sub(r'\n',r'\. ',text)
    text = re.sub(r'^','Start:',text)
    text = re.sub(r'::',r':',text)

In [ ]:
def column_cleanup(text):
    #Optimise for tokenisation eg full stops followed by a number need to change so add a Letter before the number
    #vector<-gsub("\\.\\s*(\\d)","\\.T\\1",vector)
  
    #Get rid of convert anything that has a full stop in the middle into a new line eg line .Ever
    #vector<-gsub("([A-Za-z]\\s*)\\.(\\s*[A-Za-z])","\\1\n\\2",vector)
  
  
    #vector<-gsub("([A-Za-z]+.*)\\?(.*[A-Za-z]+.*)","\\1 \\2",vector)
  
    #Convert word return to space
    text = re.sub(r'\r\n',r' ',text)
    text = re.sub(r'\r',r'\n',text)
      
  
  
    #Convert ,hi to fullstop the the word if it is a capital letter
    text = re.sub(r',([A-Z])',r'\n\1',text)
    
    #Conver "., or . ,"  to a space and vice versa
    ##@Leo: Here for . I will check if there is a number before the ., if so, the dot will be kept, such as 2.5mg, 
    # otherwise, it will be replaced with ". "
    text = re.sub(r'(\s+\.)', r'. ', text)
    text = re.sub(r'(\s+\,)', r', ', text)
    text = re.sub(r'(\.\s*\,)|(\s*\,\s*\.)|(^/d(\.\s*)+)', r'. ', text)
    #vector = stri_replace_all(vector,"\\.",regex="(\\.\\s*\\,)|(\\,\\s*\\.)|((\\.\\s*)+)")
    #vector<-gsub("\\.\\s*\\,","\\.",vector)
    #vector<-gsub("\\,\\s*\\.","\\.",vector)
    #vector<-gsub("(\\.\\s*)+","\\.",vector)
    
    #querySebestian: what is this
    #Get rid of middle of line newlines which seems to
    #happen e.g. I am Sebastian and \n I live in a hole
    #vector = re.sub("(?:\\h*\\R)++(?!\\z)\\h*", " ", vector, perl=Ture)

    #Get rid of ASCCII hex here
    text = re.sub(r"\\[Xx].*?\\", " ", text)
    
    #querySebastian: why? Hello?... Hello?...k -> Hello ...k
    #Get rid of query type punctuation:
    text = re.sub(r"(.*)\?(.*[A-Za-z]+)",r"\1 \2",text)

    #Get rid of pointless single quote marks
    text = re.sub(r"'",r"",text)
    text_list = list(map(lambda x: sent_tokenize(x),text.split('\n')))
    #print(text_list)
    text = ' '.join([item for sublist in text_list for item in sublist])
    
    return text
    

In [ ]:
list(map(lambda x: sent_tokenize(column_cleanup(x)),example.split('\n')))

In [ ]:
example.split('\n')

In [4]:
column_cleanup(example)

NameError: name 'column_cleanup' is not defined

In [5]:
"hospital: random nhs foundation trust hospital number: j6044658 patient name:  jargon, victoria general practitioner: dr.martin, marche date of procedure:  2009-11-11 endoscopist: dr.sullivan, shelby second endoscopist: dr.al-basha, mahfoodha medications: fentanyl  12.5mcg midazolam  6mg instrument:  fg5 extent of exam:  goj indications: follow-up ulcer healing procedure performed: gastroscopy (ogd) findings:  no evidence of barretts oesophagus, short 2 cn hiatus hernia.oesophageal biopsies taken from three levels as requested.ogd today to assess for ulceration/ongoing bleeding.diaphragmatic pinch:40cm .she has a small hiatus hernia .we will re-book for 2 weeks, rebanding.tiny erosions at the antrum.biopsies taken from top of stricture-metal marking clips in situ.the varices flattened well with air insufflation.he is on barretts screeling list in october 2017 at st thomas.halo 90 done with good effect endoscopic diagnosis:  post chemo-radiotherapy stricture"

'hospital: random nhs foundation trust hospital number: j6044658 patient name:  jargon, victoria general practitioner: dr.martin, marche date of procedure:  2009-11-11 endoscopist: dr.sullivan, shelby second endoscopist: dr.al-basha, mahfoodha medications: fentanyl  12.5mcg midazolam  6mg instrument:  fg5 extent of exam:  goj indications: follow-up ulcer healing procedure performed: gastroscopy (ogd) findings:  no evidence of barretts oesophagus, short 2 cn hiatus hernia.oesophageal biopsies taken from three levels as requested.ogd today to assess for ulceration/ongoing bleeding.diaphragmatic pinch:40cm .she has a small hiatus hernia .we will re-book for 2 weeks, rebanding.tiny erosions at the antrum.biopsies taken from top of stricture-metal marking clips in situ.the varices flattened well with air insufflation.he is on barretts screeling list in october 2017 at st thomas.halo 90 done with good effect endoscopic diagnosis:  post chemo-radiotherapy stricture'

In [ ]:
def spell_check(pattern, replacement, x, fixed = False):
    distance_list = []
    for pattern_word in pattern:
        distance_list.append(editdistance.eval(x, pattern_word))
    

In [16]:
def dict_replacement(dictionary, string):
    for key, val in dictionary.items():
        string = re.sub(key,val,string) 
    return string

In [18]:
dictionary = {'he':'eh', 'lo':'ol'}

In [19]:
dict_replacement(dictionary, 'hello')

'ehlol'

In [15]:
x

'aellb'

In [62]:
from nltk import sent_tokenize

In [77]:
re.sub(r'(\.\s*\,)|(\,\s*\.)|(^/d(\.\s*)+)', r'. ', example)#[2]

"Hospital: Random NHS Foundation Trust\nHospital Number: J6044658\nPatient Name:  Jargon, Victoria\nGeneral Practitioner: Dr. Martin, Marche\nDate of procedure:  2009-11-11\nEndoscopist: Dr. Sullivan, Shelby\nSecond endoscopist: Dr. al-Basha, Mahfoodha\nMedications: Fentanyl  12.5mcg\nMidazolam  6mg\nInstrument:  FG5\nExtent of Exam:  GOJ\nIndications: Follow-up ULCER HEALING\nProcedure Performed: Gastroscopy (OGD)\nFindings:  No evidence of Barrett's oesophagus, short 2 cn hiatus hernia. Oesophageal biopsies taken from three levels as requested. OGD today to assess for ulceration/ongoing bleeding. Diaphragmatic pinch:40cm . She has a small hiatus hernia . We will re-book for 2 weeks, rebanding. Tiny erosions at the antrum. Biopsies taken from top of stricture-metal marking clips in situ. The varices flattened well with air insufflation. He is on Barrett's Screeling List in October 2017 at St Thomas'.\nHALO 90 done with good effect\nEndoscopic Diagnosis:  Post chemo-radiotherapy strict

In [74]:
example

"Hospital: Random NHS Foundation Trust\nHospital Number: J6044658\nPatient Name:  Jargon, Victoria\nGeneral Practitioner: Dr. Martin, Marche\nDate of procedure:  2009-11-11\nEndoscopist: Dr. Sullivan, Shelby\nSecond endoscopist: Dr. al-Basha, Mahfoodha\nMedications: Fentanyl  12.5mcg\nMidazolam  6mg\nInstrument:  FG5\nExtent of Exam:  GOJ\nIndications: Follow-up ULCER HEALING\nProcedure Performed: Gastroscopy (OGD)\nFindings:  No evidence of Barrett's oesophagus, short 2 cn hiatus hernia.,Oesophageal biopsies taken from three levels as requested.,OGD today to assess for ulceration/ongoing bleeding.,Diaphragmatic pinch:40cm .,She has a small hiatus hernia .,We will re-book for 2 weeks, rebanding.,Tiny erosions at the antrum.,Biopsies taken from top of stricture-metal marking clips in situ.,The varices flattened well with air insufflation.,He is on Barrett's Screeling List in October 2017 at St Thomas'.\nHALO 90 done with good effect\nEndoscopic Diagnosis:  Post chemo-radiotherapy strict

In [22]:
from pyensae.languages import r2python

ModuleNotFoundError: No module named 'pyensae'